# Maps

In [13]:
%%HTML
<style>
.container{width:75% !important;}
.text_cell_rendered_html{width:20% !important;}
</style>

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf

import cartopy
from cartopy import crs as ccrs

from bokeh.tile_providers import STAMEN_TONER
from bokeh.models import WMTSTileSource

hv.notebook_extension('bokeh')

/home/kalidus/github/holoviews/holoviews/core/util.py:23: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  datetime_types = datetime_types + (pd.tslib.Timestamp,)
/home/kalidus/anaconda3/lib/python3.5/site-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The Vega10 colormap was deprecated in version 2.0. Use tab10 instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)
/home/kalidus/anaconda3/lib/python3.5/site-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The Vega20b colormap was deprecated in version 2.0. Use tab20b instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)
/home/kalidus/anaconda3/lib/python3.5/site-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The Vega10_r colormap was deprecated in version 2.0. Use tab10_r instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)
/home/kalidus/anaconda3/lib/python3.5/site-packages/ma

In [3]:
from bokeh.models import (
    GeoJSONDataSource,
    HoverTool,
    LinearColorMapper
)
from bokeh.plotting import figure
from bokeh.palettes import Viridis6
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [4]:
data = pd.read_csv("data/listings/08042017/listings.csv")   #('listings_tot.csv')
num_hogares = pd.read_csv("data/num_hogares/NumHogaresYFamilias2011.csv", sep=";")
num_hogares.columns = num_hogares.columns.str.lstrip()

def drop_digits(in_str):
    digit_list = "1234567890"
    for char in digit_list:
        in_str = in_str.str.replace(char, "")

    return in_str

num_hogares.Barrio = drop_digits(num_hogares.Barrio)
num_hogares.Barrio = num_hogares.Barrio.str.replace(".", "")

n_airbnbs_barri= data.neighbourhood_cleansed.value_counts()
n_airbnbs_barri = n_airbnbs_barri[n_airbnbs_barri.index.isin(num_hogares.Barrio)]

num_hogares = num_hogares[num_hogares.Barrio.isin(n_airbnbs_barri.index)]
num_hogares.index = num_hogares.Barrio
num_hogares.drop("Barrio", axis=1)[:2]

num_hogares = num_hogares.NumHogares
#num_hogares.drop(["Can Peguera", "Baró de Viver", "Torre Baró", "Vallbona"], inplace=True)#outliers
num_hogares=num_hogares * 1000 #ugly hack to get rid of the "." decimal
density = n_airbnbs_barri/num_hogares

/home/kalidus/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
barri_json_path = r"data/divisiones_administrativas/barris/barris_geo.json"
with open(barri_json_path, 'r') as f:
    geo_source = GeoJSONDataSource(geojson=f.read())

color_mapper = LinearColorMapper(palette=Viridis6)
TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

p = figure(title="barris", tools=TOOLS, x_axis_location=None, y_axis_location=None, width=800, height=800)
p.grid.grid_line_color = None

p.patches('xs', 'ys', fill_alpha=0.7, 
          line_color='white', line_width=1, source=geo_source)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [("barri:", "@N_Barri")]

#output_file("PBIar.html", title="Testing islands in bokeh")
show(p)

In [6]:
density = pd.DataFrame({"N_Barri":density.index, "value":density.values})

In [7]:
density.value = density.value * 100
density['log_value'] =  np.log(density.value)

In [8]:
shapefile = "data/divisiones_administrativas/barris/shape/barris_geo.shp"
shapes = cartopy.io.shapereader.Reader(shapefile)
density_hv = hv.Dataset(density)

density_hv.data.dropna(inplace=True)

In [15]:
%%opts Overlay [width=1200 height=800 xaxis=None yaxis=None] 
%%output filename="holoviewsmap"
myhover =  HoverTool(tooltips = """
            <div>
                <img src=@path width="170" height="170"></img>
            </div> 
            <div>
                <span style="font-size: 12px; font-weight: bold;">@log_value</span>
            </div>        
                """
                    )


gv.Shape.from_records(shapes.records(), density_hv, on='N_Barri', value='value',
                      index=['N_Barri', 'log_value'], crs=ccrs.PlateCarree(), group="Densitat Airbnb Barcelona per nombre d'hogars",
                      drop_missing=False)
%%opts Shape (cmap='Reds') [tools=['hover'] width=1200 height=1200 colorbar=True toolbar='above' xaxis=None yaxis=None]

#gv.WMTS(WMTSTileSource(url='https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'))

/home/kalidus/anaconda3/lib/python3.5/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: webgl was deprecated in Bokeh 0.12.6 and will be removed, use output_backend instead.
  warn(message)
/home/kalidus/anaconda3/lib/python3.5/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: webgl was deprecated in Bokeh 0.12.6 and will be removed, use output_backend instead.
  warn(message)
/home/kalidus/anaconda3/lib/python3.5/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: webgl was deprecated in Bokeh 0.12.6 and will be removed, use output_backend instead.
  warn(message)
/home/kalidus/anaconda3/lib/python3.5/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: webgl was deprecated in Bokeh 0.12.6 and will be removed, use output_backend instead.
  warn(message)


b':NdOverlay   [Index,N_Barri,log_value]\n   :Shape   [Longitude,Latitude]   (value)'

AttributeError: 'Points' object has no attribute 'opts'